In [46]:
from neuromaps import datasets, images, nulls, resampling,stats
neurosynth = datasets.fetch_annotation(source='neurosynth')
abagen = datasets.fetch_annotation(source='abagen')
neurosynth, abagen = resampling.resample_images(neurosynth, abagen, 'MNI152', 'fsaverage')
rotated = nulls.alexander_bloch(neurosynth, atlas='fsaverage', density='10k',
                                n_perm=100, seed=1234)
print(rotated.shape)


(20484, 100)


# Nulls with parcellated data

null模型函数也可以处理分割的数据，并通过接受一个额外的可选关键字参数来实现：parcellation。如果提供了这个参数，null函数会假定这是一个与所提供的数据处于同一空间的图像元组（左、右半球，如常）

一般来说，你已经有了预分割的数据，但为了演示，我们将使用nilearn获取一个表面分化（为10k fsaverage系统）

In [47]:
from nilearn.datasets import fetch_atlas_surf_destrieux
from neuromaps import datasets, images, nulls, resampling
destrieux = fetch_atlas_surf_destrieux()
print(sorted(destrieux))

['description', 'labels', 'map_left', 'map_right']


In [48]:
# 左脑和右脑都是ndarray
print(type(destrieux['map_left']))
print(len(destrieux['map_left']), len(destrieux['map_right']))

<class 'numpy.ndarray'>
10242 10242


In [37]:
print(len(destrieux['labels']))

76


不幸的是，此处提供的 Destrieux 地图集设计为左右半球具有相同的标签值。神经图中处理分块数据的函数始终假设标签 ID 在半球之间递增。此外，map_left 和 map_right 值是简单的 numpy 数组，neuromaps 更喜欢使用 GIFTI 图像。我们可以将数组转换为 GIFTI 标签图像，然后重新标记它们，以便标签在半球之间是连续的

请注意，我们在调用relabel_gifti()时设置了background=['Medial_wall']。这是因为，默认情况下，内侧壁的标签是42，而我们希望它被设置为0。（神经图谱函数假定0标签是背景，在大多数计算中它被省略了）

In [57]:
labels = [label.decode() for label in destrieux['labels']]
parc_left = images.construct_shape_gii(destrieux['map_left'], labels=labels,
                                       intent='NIFTI_INTENT_LABEL')
parc_right = images.construct_shape_gii(destrieux['map_right'], labels=labels,
                                        intent='NIFTI_INTENT_LABEL')
parcellation = images.relabel_gifti((parc_left, parc_right), background=['Medial_wall'])
print(parcellation)         

(<nibabel.gifti.gifti.GiftiImage object at 0x1773fce50>, <nibabel.gifti.gifti.GiftiImage object at 0x1773e4640>)


In [64]:
from nibabel.gifti import giftiio
import nibabel
nibabel.save(parc_left,"left.gii")

In [65]:
nibabel.save(parc_right,"right.gii")

In [70]:
# images.load_data(nibabel.load("left.gii")).shape
images.load_data(parcellation[0]).shape


(10242,)

我们可以使用这些图像，用neuromaps.parcellate.Parcellater类的一个实例来分割我们的数据

In [40]:
from neuromaps import parcellate
destrieux = parcellate.Parcellater(parcellation, 'fsaverage').fit()
neurosynth_parc = destrieux.transform(neurosynth, 'fsaverage')
abagen_parc = destrieux.transform(abagen, 'fsaverage')
print(neurosynth_parc.shape, abagen_parc.shape)

(148,) (148,)


In [72]:
neurosynth_parc


array([-1.3502014e-03,  5.1559869e-04,  2.3535038e-03,  2.2456176e-03,
       -1.1681822e-03, -2.8523915e-03,  1.4707146e-03,  1.1858679e-03,
       -3.0711740e-03, -2.8664269e-03,  2.4696981e-04,  1.0797515e-03,
       -2.9965201e-03, -2.6258340e-03, -1.2355103e-03,  2.7996144e-04,
       -1.3259611e-04, -3.2476193e-04,  2.8039730e-04,  1.1730352e-03,
       -6.2648574e-04, -2.5964313e-04, -3.0956690e-03, -2.0425867e-03,
       -2.2414706e-03,  2.3510500e-03,  3.3410650e-03,  3.2120552e-03,
        4.1632741e-03, -1.0691602e-03, -2.2302552e-03, -2.7986723e-03,
        2.0216980e-03,  6.2060350e-04, -2.0926539e-03,  2.5778895e-03,
       -9.7296183e-04, -1.4641039e-03, -1.8901137e-03, -1.0817513e-03,
        2.2054312e-03,  4.0947236e-04, -1.7860460e-03, -4.9383385e-04,
        3.0656324e-03, -3.2452718e-04, -2.8216566e-03, -4.3447549e-04,
        7.7097403e-04, -1.7246705e-03, -6.5903686e-04, -1.1652646e-03,
       -8.5511559e-04, -3.0647888e-04, -2.0415140e-03,  1.3204279e-03,
      

In [44]:
images.relabel_gifti((image1,image2))

(<nibabel.gifti.gifti.GiftiImage at 0x17797a070>,
 <nibabel.gifti.gifti.GiftiImage at 0x177979b80>)

*现在我们已经有了我们的分割数组，我们可以生成我们的空映射。我们使用与上面相同的调用，但提供额外的分割参数*

In [56]:
neurosynth_parc.shape

(148,)

In [73]:
rotated = nulls.alexander_bloch(neurosynth_parc, atlas='fsaverage', density='10k',
                                n_perm=100, seed=1234, parcellation=parcellation)


In [79]:
rotated

array([[-2.2414706e-03, -2.2414706e-03,  2.3510500e-03, ...,
         1.3004806e-03, -2.6258340e-03,  3.9832033e-03],
       [-2.1700652e-03, -3.0956690e-03, -4.9383385e-04, ...,
         9.8767364e-04, -6.5903686e-04, -2.0246899e-03],
       [-3.0711740e-03, -1.1652646e-03, -2.9965201e-03, ...,
        -1.7860460e-03,  2.3535038e-03, -2.9611134e-03],
       ...,
       [-1.6065268e-03, -2.0954341e-03,  1.7101986e-04, ...,
         2.6276954e-03, -6.3403806e-04, -2.1439223e-03],
       [-2.4768854e-03, -3.3256172e-03, -1.1604582e-03, ...,
         6.1012350e-04,  3.6375511e-03, -1.7169325e-03],
       [-6.2517723e-04, -9.0806244e-04,  1.9060864e-05, ...,
         1.9060864e-05,  3.0768814e-03, -6.2517723e-04]], dtype=float32)

In [55]:
rotated.shape

(20484, 100)

我们可以像以前一样将生成的数组传递给compare_images()的nulls参数

In [43]:
corr, pval = stats.compare_images(neurosynth_parc, abagen_parc, nulls=rotated)
print(f'r = {corr:.3f}, p = {pval:.3f}')

r = 0.416, p = 0.376
